In [2]:
import pandas as pd
import numpy as np
import json
import glob
%run twokenize.py

In [4]:
testDir = '../../data/external/trump-bots/'
botDir = '../../data/external/botresults/'
outDir = '../../data/processed/bot-tweets/'

In [5]:
# Crudely combine
process = []
for f in glob.glob((botDir + "*.txt")):
    with open(f, "rb") as infile:
        for line in infile:
            process.append(json.loads(line))
raw = pd.DataFrame.from_records(process)

print (raw.shape)
raw.head()

(77722, 33)


,contributors,coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,...,quoted_status_id,quoted_status_id_str,retweet_count,retweeted,retweeted_status,source,text,timestamp_ms,truncated,user
0,None,None,Mon Jul 06 00:12:43 +0000 2015,NaN,"{u'user_mentions': [{u'indices': [3, 17], u'sc...",{u'media': [{u'source_status_id_str': u'617821...,NaN,0,False,low,...,NaN,NaN,0,False,"{u'contributors': None, u'truncated': False, u...","<a href=""http://twitter.com/download/android"" ...",RT @robinleeclark: Hillary is a murderer and a...,1436141563046,False,"{u'follow_request_sent': None, u'profile_use_b..."
1,None,None,Mon Jul 06 00:17:23 +0000 2015,NaN,"{u'user_mentions': [{u'indices': [3, 15], u'sc...",NaN,NaN,0,False,low,...,NaN,NaN,0,False,"{u'contributors': None, u'truncated': False, u...","<a href=""http://twitter.com/download/android"" ...",RT @PatriotMash: Clinton uses two aides and a ...,1436141843293,False,"{u'follow_request_sent': None, u'profile_use_b..."
2,None,None,Mon Jul 06 00:19:58 +0000 2015,NaN,"{u'user_mentions': [{u'indices': [3, 15], u'sc...",{u'media': [{u'source_status_id_str': u'617673...,NaN,0,False,low,...,NaN,NaN,0,False,"{u'contributors': None, u'truncated': False, u...","<a href=""http://twitter.com/download/android"" ...",RT @NoahWehrman: Press prepare for another day...,1436141998259,False,"{u'follow_request_sent': None, u'profile_use_b..."
3,None,None,Mon Jul 06 00:20:15 +0000 2015,NaN,"{u'user_mentions': [], u'symbols': [], u'trend...",NaN,NaN,0,False,low,...,NaN,NaN,0,False,NaN,"<a href=""http://ifttt.com"" rel=""nofollow"">IFTT...",nytimes: Hillary Clinton reassures gay youth i...,1436142015542,False,"{u'follow_request_sent': None, u'profile_use_b..."
4,None,None,Mon Jul 06 00:20:17 +0000 2015,NaN,"{u'user_mentions': [{u'indices': [3, 18], u'sc...",NaN,NaN,0,False,low,...,NaN,NaN,0,False,"{u'contributors': None, u'truncated': False, u...","<a href=""http://twitter.com/download/android"" ...",RT @LessGovMoreFun: Truth be told: https://t....,1436142017984,False,"{u'follow_request_sent': None, u'profile_use_b..."


In [6]:
def tknz(text):
    tokens = tokenizeRawTweetText(text)
    filt = [x for x in tokens if not (x.startswith('RT') 
                                   or x.startswith('@') or x.startswith(':') 
                                   or x.startswith('https:') or x.startswith('-') 
                                     or x.startswith('.') or x.startswith(','))]
    return filt

def hsh(tokens):
    # reads a list of tokens
    tuped = tuple(tokens)
    hashed = hash(tuped)
    return hashed

In [7]:
def extractInfo(tweet):
    # User variables
    userID = tweet['user']['id_str']
    screenName = tweet['user']['screen_name']
    # Tweet Variables
    text = tweet['text']
    tokens = tknz(text)
    hashed = hsh(tokens)
    
    timestamp = tweet['timestamp_ms']
    
    return {'userID':userID, 'screenName':screenName, 'timestamp':timestamp, \
            'text':text, 'tokens':tokens, 'hash':hashed}
    
    #### below is a horrible rendition of network variables
    
"""
# RT, reply, quoting, or none?
    try:
        if (tweet['retweeted_status'] == True):
            method = 'retweet'
            otherID = tweet['retweeted_status']['user']['id_str']

        if (type(tweet['in_reply_to_user_id_str']) != np.float64):
            method = 'replyUser'
            otherID = tweet['in_reply_to_user_id_str']
        elif (tweet['in_reply_to_status_id_str']):
            method = 'replyStatus'
            otherID = None
        else:
            method = None
            otherID = None

        return {'userID':userID, 'screenName':screenName, 'text':text, 'timestamp':timestamp,
                'method':method, 'otherID':otherID}
    except:
        print tweet
"""

"\n# RT, reply, quoting, or none?\n    try:\n        if (tweet['retweeted_status'] == True):\n            method = 'retweet'\n            otherID = tweet['retweeted_status']['user']['id_str']\n\n        if (type(tweet['in_reply_to_user_id_str']) != np.float64):\n            method = 'replyUser'\n            otherID = tweet['in_reply_to_user_id_str']\n        elif (tweet['in_reply_to_status_id_str']):\n            method = 'replyStatus'\n            otherID = None\n        else:\n            method = None\n            otherID = None\n\n        return {'userID':userID, 'screenName':screenName, 'text':text, 'timestamp':timestamp,\n                'method':method, 'otherID':otherID}\n    except:\n        print tweet\n"

In [8]:
extracted = pd.DataFrame.from_records(raw.apply(lambda x: extractInfo(x), axis=1))
extracted.head()

,hash,screenName,text,timestamp,tokens,userID
0,-2184387373131684931,Scrufey21,RT @robinleeclark: Hillary is a murderer and a...,1436141563046,"[Hillary, is, a, murderer, and, a, traitor, an...",490179072
1,1607247562242048296,Scrufey21,RT @PatriotMash: Clinton uses two aides and a ...,1436141843293,"[Clinton, uses, two, aides, and, a, rope, to, ...",490179072
2,-187648859270779684,Scrufey21,RT @NoahWehrman: Press prepare for another day...,1436141998259,"[Press, prepare, for, another, day, of, coveri...",490179072
3,-7462846675793459304,CyberAnonymous,nytimes: Hillary Clinton reassures gay youth i...,1436142015542,"[nytimes, Hillary, Clinton, reassures, gay, yo...",128334973
4,-289237171859682731,Scrufey21,RT @LessGovMoreFun: Truth be told: https://t....,1436142017984,"[Truth, be, told]",490179072


In [9]:
extracted['timestamp'] = pd.to_datetime(extracted['timestamp'], unit='ms')

# Get most popular 
print ('Most popular tweet hashes.')
extracted['hash'].value_counts().head(10)

Most popular tweet hashes.


 717126146592199451     1423
 5162929327475364277    1362
-4501194672645902651     807
-2692256532814125541     624
 3527539                 578
 7510515497875988663     520
-4907917567665423769     484
 2314862951965282667     176
 8969368713954905093     146
-7003498234340419358     142
Name: hash, dtype: int64

In [11]:
tweetsByUsers = extracted[['userID', 'hash']].groupby('hash')\
    .agg({'userID': pd.Series.nunique})\
    .sort_values(by='userID', ascending=False)
    
print ('Tweets with multiple (unique) users.')
tweetsByUsers.head()

Tweets with multiple users.


,userID
hash,
-2928380469476387809,50
8969368713954905093,50
191154071824568122,48
2378718859638583145,47
3527539,19


In [12]:
hashes = extracted[['screenName', 'text', 'tokens', 'hash']].set_index('hash')
hashes.head()

,screenName,text,tokens
hash,,,
-2184387373131684931,Scrufey21,RT @robinleeclark: Hillary is a murderer and a...,"[Hillary, is, a, murderer, and, a, traitor, an..."
1607247562242048296,Scrufey21,RT @PatriotMash: Clinton uses two aides and a ...,"[Clinton, uses, two, aides, and, a, rope, to, ..."
-187648859270779684,Scrufey21,RT @NoahWehrman: Press prepare for another day...,"[Press, prepare, for, another, day, of, coveri..."
-7462846675793459304,CyberAnonymous,nytimes: Hillary Clinton reassures gay youth i...,"[nytimes, Hillary, Clinton, reassures, gay, yo..."
-289237171859682731,Scrufey21,RT @LessGovMoreFun: Truth be told: https://t....,"[Truth, be, told]"


In [24]:
tweets = raw[raw['retweeted'] == False]
retweets = raw[raw['retweeted'] == True]

In [25]:
raw['entities'].head()

0    {u'user_mentions': [{u'indices': [3, 17], u'sc...
1    {u'user_mentions': [{u'indices': [3, 15], u'sc...
2    {u'user_mentions': [{u'indices': [3, 15], u'sc...
3    {u'user_mentions': [], u'symbols': [], u'trend...
4    {u'user_mentions': [{u'indices': [3, 18], u'sc...
Name: entities, dtype: object